<h2> 3.6 Featurizing text data with tfidf weighted word-vectors </h2>

* We take the questions from Q1 & Q2 and apply tf idf on them.
* Get the word pool and their corresponding tf-idf score in a dictionary.

```
##Code to increase the ram of colab

d=[]
while(1):
  d.append('1')

## For tutorial on google colab exploitation

#https://medium.com/@oribarel/getting-the-most-out-of-your-google-colab-2b0585f82403
```



In [0]:
!python -m spacy download en_core_web_lg
#after this step restart the runtime for en_core_web_lg to load

In [5]:
!pip install fastparquet
!pip install p_tqdm
!pip install swifter

     |████████████████████████████████| 153kB 3.2MB/s 
     |████████████████████████████████| 61kB 3.8MB/s 
  Created wheel for fastparquet: filename=fastparquet-0.3.3-cp36-cp36m-linux_x86_64.whl size=253680 sha256=45dc61f9873c1498a611aa238f2677409818e1b1bea267f6c74ed7b41eda6e2f
  Stored in directory: /root/.cache/pip/wheels/a0/27/9f/d8066bbbbb77e97d8ad3daf4de155ead73693bc4aa2f52098c
  Created wheel for thrift: filename=thrift-0.13.0-cp36-cp36m-linux_x86_64.whl size=345239 sha256=6659327b089bb9db1e9a693d2677d281ea76f2075515f4078390063e4d032730
  Stored in directory: /root/.cache/pip/wheels/02/a2/46/689ccfcf40155c23edc7cdbd9de488611c8fdf49ff34b1706e
Successfully built fastparquet thrift
     |████████████████████████████████| 163kB 3.3MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 112kB 53.6MB/s 
  Created wheel for p-tqdm: filename=p_tqdm-1.3.3-cp36-none-any.whl size=3989 sha256=ba4a3961cb081d42349a096174658d29dad31db133618f3cc7999d

In [6]:
!pip install -U ipykernel

     |████████████████████████████████| 122kB 3.5MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.2.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement tornado~=4.5.0, but you'll have tornado 6.0.4 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


* Importing the already saved files.

In [0]:
import pandas as pd
import numpy as np
import re
import sys
import os
import warnings
import swifter

from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
warnings.filterwarnings('ignore')
import spacy
from p_tqdm import p_map

In [0]:
if os.path.isfile('/content/drive/My Drive/Colab Notebooks/Models/Quora/qstn_train_basic_ft.parquet.gzip'):
  qstns_train_ppro = pd.read_parquet('/content/drive/My Drive/Colab Notebooks/Models/Quora/qstn_train_basic_ft.parquet.gzip', engine='fastparquet')
else:
  print("download qstn_train_basic_ft.parquet.gzip from drive or run previous notebook")

if os.path.isfile('/content/drive/My Drive/Colab Notebooks/Models/Quora/qstn_train_adv_ft.parquet.gzip'):
  qstns_train_nlp = pd.read_parquet('/content/drive/My Drive/Colab Notebooks/Models/Quora/qstn_train_adv_ft.parquet.gzip', engine='fastparquet')
else:
  print("download qstn_train_adv_ft.parquet.gzip from drive or run previous notebook")

In [3]:
#Reading the data from kaggle

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!kaggle -v
  
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"vinodkumar9576","key":"af074010bd4c7bb75f9d1fe71f31452d"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c quora-question-pairs

# To find the list of file names in zip file
#for name in zip_file.namelist():
#    print ('%s' % (name))
#zip_file.close()

#Found this code snipper for reading files in a zipfile within a zipfile.
#https://stackoverflow.com/questions/11930515/unzip-nested-zip-files-in-python?noredirect=1&lq=1

import zipfile
from io import BytesIO
import re

with zipfile.ZipFile('/content/quora-question-pairs.zip', "r") as z:
  with z.open('train.csv.zip') as z2:
    z2_filedata = BytesIO(z2.read())
    with zipfile.ZipFile(z2_filedata) as nested_zip:
      qstns_train = pd.read_csv(nested_zip.open('train.csv'))
  with z.open('test.csv.zip') as z3:
    z3_filedata = BytesIO(z3.read())
    with zipfile.ZipFile(z3_filedata) as nested_zip2:
      qstns_test = pd.read_csv(nested_zip2.open('test.csv'),dtype = {"test_id":object,"question1":object,"question2":object})

#print("qstns_train :",end='\n')
#print(qstns_train.head(2))
#print("qstns_test :",end='\n')
#print(qstns_test.head(2))

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.4MB 3.2MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 2.0 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=f6758a3ca51f69f092152abe5480e40abf75049c9b4aba11013d63825ca7ff3f
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.6/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", line 149, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure

* Q1 & Q2 columns stored as object, and tfidf vector accepts i/p as string so we are going to chnage each question to string.

#TFIDF weighted W2V

* Making pool of words with tfidf score from the original dataset.

* Making the questions from Q1 & Q2

* Forming dictionary with words after TfidfVector & tfidf scores

In [4]:
%%time
qstns_train['question1']=qstns_train['question1'].swifter.apply(lambda x: str(x))
qstns_train['question2']=qstns_train['question2'].swifter.apply(lambda x: str(x))


CPU times: user 1.49 s, sys: 12.6 ms, total: 1.5 s
Wall time: 1.49 s


In [0]:
#making antother data frame
qstrain_tfidf_wtd_w2v = qstns_train.copy()

In [0]:
questions = list(qstrain_tfidf_wtd_w2v['question1']) + list(qstrain_tfidf_wtd_w2v['question2'])

model_tfidf = TfidfVectorizer(lowercase=False,)
model_tfidf.fit_transform(questions)

tfidf_vect = dict(zip(model_tfidf.get_feature_names(),model_tfidf.idf_))

* Now we convert each question to avg word to vec.
* We use GLOVE model which comes in "Spacy".
https://spacy.io/usage/vectors-similarity
* GLOVE model is trained on Wikipedia and therefore, it is stronger in terms of word semantics.

In [0]:
import spacy
nlp = spacy.load('en_core_web_lg')
#above spacy model provides max of 300 dim vector of a word
# so we make our tfidf weighted w2v model text processing to no. of questns X 300.

* Instead of running above which is serial processing, we use parallel processing to reduce the time.

In [0]:
#Function for calculating avg Tfidf Weighted W2V
def txt2vec_q1 (df):
  vecs1 = []
  for qstn1 in tqdm(list(df), position=0, desc ='Question'):
    doc1 = nlp(qstn1) #word2vec conversion using
    mean_vec1 = np.zeros([len(doc1),300])
    #making array of zeros of size of question
    for word1 in doc1:
      vec1 = word1.vector #getting the vector of each word in question

      try:
        idf = tfidf_vect[str(word1)]
      except:
        idf = 0
    
      mean_vec1 +=vec1*idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
  
  return vecs1

In [0]:
q1_txt2vec = p_map(txt2vec_q1, qstrain_tfidf_wtd_w2v['question1'])
#above we get as list but shape will be like 1 X rows in qstns
#(each row contains 300 valued array)

q1_txt2vec_arr = np.array(q1_txt2vec)

# we reshaped the list to rows of qstns_train X 1 column
qstrain_tfidf_wtd_w2v['q1_feats_m'] = list(q1_txt2vec_arr.reshape(q1_txt2vec_arr.shape[1],q1_txt2vec_arr.shape[2]))

#It took alomost 1 hour to complete the above job

KeyboardInterrupt: ignored

In [0]:
# avg tfidf weighted vector of question2

def txt2vec_q2(df):
  vecs2 = []
  for qstn2 in tqdm(list(qstrain_tfidf_wtd_w2v['question2']), position=0, desc ='Question'):
    doc2 = nlp(qstn2)
    mean_vec2 = np.zeros([len(doc2),300])
    for word2 in doc2:
      vec2=word2.vector
      try:
        idf = tfidf_vect[str(word2)]
      except:
        idf = 0
    
      mean_vec2 += vec2*idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
  return vecs2

In [0]:
from p_tqdm import p_map
q2_txt2vec = p_map(txt2vec_q2, qstrain_tfidf_wtd_w2v['question2'])
#above we get as list but shape will be like 1 X rows in qstns
#(each cell contains 300 valued array)

q2_txt2vec_arr = np.array(q2_txt2vec)

# we reshaped the list to rows of qstns_train X 1 column
qstrain_tfidf_wtd_w2v['q2_feats_m'] = list(q2_txt2vec_arr.reshape(q2_txt2vec_arr.shape[1],q2_txt2vec_arr.shape[2]))

print('q2_features computation completed')

# Storing the above data into file to avoid rerunning
qstrain_tfidf_wtd_w2v.to_parquet('/content/drive/My Drive/Colab Notebooks/Models/Quora/qstns_train_q2_feats_m.parquet.gzip',compression='gzip')

Question: 100%|██████████| 404290/404290 [1:06:22<00:00, 101.53it/s]


In [0]:
qstrain_tfidf_wtd_w2v.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


* importing nlp_featured file from drive
* importing preprocessed_features_train

* Now we are getting nlp features, and features from preprocessing also extra features from above processed dataframe.

In [0]:
if os.path.isfile('/content/drive/My Drive/Colab Notebooks/Models/Quora/nlp_feat.parquet.gzip'):
  nlp_feat = pd.read_parquet('/content/drive/My Drive/Colab Notebooks/Models/Quora/nlp_feat.parquet.gzip',engine='fastparquet')
else:
  nlp_feat = qstns_train_nlp.drop(['qid1','qid2','question1','question2'], axis=1)

if os.path.isfile('/content/drive/My Drive/Colab Notebooks/Models/Quora/ppro_feat.parquet.gzip'):
  ppro_feat = pd.read_parquet('/content/drive/My Drive/Colab Notebooks/Models/Quora/ppro_feat.parquet.gzip', engine ='fastparquet')
else:
  ppro_feat = qstns_train_ppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)

if os.path.isfile('/content/drive/My Drive/Colab Notebooks/Models/Quora/txt_ft_q1.parquet.gzip'):
  txt_ft_q1 = pd.read_parquet('/content/drive/My Drive/Colab Notebooks/Models/Quora/txt_ft_q1.parquet.gzip', engine = 'fastparquet')
else:
  txt_vect = qstns_train.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
  #Below will split each cell of qstns_train['q1_feats_m'] into 300 further columns
  txt_ft_q1 = pd.DataFrame(txt_vect.q1_feats_m.values.tolist(), index=txt_vect.index)

if os.path.isfile('/content/drive/My Drive/Colab Notebooks/Models/Quora/txt_ft_q2.parquet.gzip'):
  txt_ft_q2 = pd.read_parquet('/content/drive/My Drive/Colab Notebooks/Models/Quora/txt_ft_q2.parquet.gzip', engine ='fastparquet')
else:
  txt_ft_q2 = pd.DataFrame(txt_vect.q2_feats_m.values.tolist(), index=txt_vect.index)

In [0]:
nlp_feat.head()

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio,freq_qid1,freq_qid2,len_q1,len_q2,n_words_q1,n_words_q2,words_Common,words_Total,word_Share,freq_q1+q2,freq_q1-q2
0,0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.916659,0.0,1.0,2.0,13.0,100,93,93,100,0.982456,1,1,66,57,14,12,10.0,10.0,0.434783,2,2
1,1,0,0.799984,0.399996,0.749981,0.749981,0.699993,0.699993,0.0,1.0,5.0,12.5,86,63,65,73,0.571429,4,1,51,88,8,13,4.0,4.0,0.200000,5,5
2,2,0,0.399992,0.333328,0.399992,0.399992,0.399996,0.399996,0.0,1.0,4.0,12.0,65,64,43,46,0.172414,1,1,73,59,14,10,4.0,4.0,0.166667,2,2
3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,4.0,9.0,29,28,26,32,0.078947,1,1,50,65,11,9,0.0,0.0,0.000000,2,2
4,4,0,0.199996,0.099999,0.999950,0.999950,0.428565,0.428565,0.0,1.0,6.0,10.0,53,44,39,53,0.162162,3,1,76,39,13,7,2.0,2.0,0.100000,4,4


In [0]:
ppro_feat.head()

,id,freq_qid1,freq_qid2,len_q1,len_q2,n_words_q1,n_words_q2,words_Common,words_Total,word_Share,freq_q1+q2,freq_q1-q2
0,0,1,1,66,57,14,12,10.0,10.0,0.434783,2,2
1,1,4,1,51,88,8,13,4.0,4.0,0.200000,5,5
2,2,1,1,73,59,14,10,4.0,4.0,0.166667,2,2
3,3,1,1,50,65,11,9,0.0,0.0,0.000000,2,2
4,4,3,1,76,39,13,7,2.0,2.0,0.100000,4,4


In [0]:
txt_ft_q1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,id
0,-5.856872,17.449559,4.862720,7.971019,20.345586,-5.514759,-4.077800,-2.820742,8.029026,146.599092,-45.501892,-3.234358,3.729830,0.254742,-0.175374,-3.568255,-6.080312,116.803297,-3.688970,7.426226,5.425185,-3.547959,-1.379188,-0.893225,-1.110629,16.353946,11.078913,-11.610769,-15.754355,4.998403,-0.993706,-4.724699,9.943273,6.704494,11.301291,-3.338042,8.395096,1.747309,7.426497,-26.311680,...,5.199206,7.998419,6.753251,4.346089,-33.859801,5.767235,-0.172016,27.004695,7.815802,-6.899303,2.874415,-6.878698,-7.277248,-5.377169,-1.564495,-2.745505,2.192525,14.977773,3.302469,0.207592,8.093244,-3.231223,8.184192,-0.614441,-13.084552,4.903100,-8.006065,4.758994,1.822565,-17.370964,5.393082,0.384676,-8.362788,-1.880290,-10.799672,-12.999799,3.225858,1.256145,16.807275,0
1,9.356103,13.098566,18.945098,-2.079594,-15.703841,-2.173409,8.969065,-20.458267,-20.674299,13.760798,-4.915949,-12.053218,-3.429222,-0.830801,21.991671,13.013338,16.124805,10.020511,-2.831793,-7.690401,17.856084,-0.389023,-7.721393,-0.938691,31.005484,2.243424,-12.566040,0.693387,-25.827244,25.383264,7.490454,-6.530129,-16.064015,11.024914,-18.271684,13.625631,-5.834146,-12.292778,-8.388683,-9.782358,...,-12.928833,0.197516,-0.199935,-8.728641,-14.564398,13.497977,13.899349,-26.234267,9.109323,-36.759636,1.438014,0.541135,20.146178,-3.560102,12.753242,-3.273333,-0.530159,-17.508985,6.050800,0.188033,-4.779012,4.783088,-9.443197,-11.163096,-9.036913,-5.421392,-4.595363,-2.503959,-0.161683,25.948247,0.603713,-10.516349,6.040723,30.476707,3.976890,-28.254610,12.613432,-7.770673,31.456654,1
2,0.909520,16.050299,-8.126856,-4.848289,-2.806190,9.752280,4.349992,-5.120332,6.785252,106.342974,-12.946087,-4.638530,0.695082,9.784824,-2.031443,-4.317213,-15.456703,104.764930,-15.553173,-10.516228,3.736023,0.525393,-8.312115,-7.287850,1.253514,-13.797112,1.027321,-12.605664,4.652425,12.600644,9.295078,8.244325,-5.084463,9.606660,-1.387407,-9.959622,-2.832281,8.699006,-2.486138,-7.864718,...,13.481948,9.606772,-6.613973,0.026433,-15.524942,-8.622154,13.168587,35.574338,-3.318328,6.095276,9.640961,-1.357213,4.544472,3.455904,1.834285,-13.312816,-3.342612,9.240566,3.222823,-8.923410,2.985248,-1.614670,12.282682,5.741256,-9.367493,11.084403,1.472286,10.879947,-9.291166,-20.942061,2.398984,8.663028,-0.654124,16.220601,-2.719094,10.485332,-1.103132,-7.290877,19.314250,2
3,-4.950745,17.098874,-15.474965,1.044680,-2.392017,-0.051889,2.650595,-8.451192,2.584123,116.184408,-9.716004,-5.455535,-8.788246,-9.185045,-3.272475,-5.011833,-6.707426,41.608225,-2.035855,8.790774,0.534490,-10.841726,4.703956,1.865227,-6.751062,1.828618,7.245328,-7.296226,11.389556,-15.133209,-16.083755,0.625838,-1.961766,-8.172698,1.681222,0.640790,11.383758,-0.461462,-14.169289,-5.028459,...,13.451178,6.898394,-6.906343,-3.753871,-7.031335,-7.248156,6.586702,8.844432,17.324413,-7.671151,-8.872585,-15.935719,-10.249489,-13.639876,2.905817,-0.103909,-7.674743,-8.539683,14.752645,8.131742,0.888286,1.465849,-2.257965,1.870978,-2.434777,8.350177,-11.499825,1.115768,4.572976,-2.551312,-4.971480,-0.478381,-1.930166,9.336016,2.574459,4.803863,-1.182989,-2.962115,3.225704,3
4,-11.520302,19.769948,-4.510997,-6.548994,-20.835286,33.663909,-30.390504,0.826553,-19.571472,84.458577,-44.555112,-1.895740,-15.577907,18.605879,-6.384102,-16.793189,20.895356,135.837219,-23.289411,-18.831285,12.076077,-14.698343,-25.067721,-4.968902,-2.066462,-20.661421,-18.602958,-5.237883,8.010809,-32.238147,7.731455,14.683179,13.562164,-16.166323,-3.977776,-10.995790,22.970306,5.925295,26.878099,-11.937297,...,-23.037509,-3.943912,24.252129,5.761628,0.748052,-11.986194,-14.955859,-29.215928,-2.999259,54.238517,-18.160312,14.010531,4.815329,-6.980127,-13.478161,-15.408554,14.5

In [0]:
txt_ft_q2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,id
0,0.398579,13.991607,-0.504564,9.254431,13.906436,-4.777694,-5.274421,-0.201208,4.940558,134.735950,-33.864003,1.395770,6.165768,1.936200,-3.983068,-5.263655,-5.013721,99.439878,-7.434865,2.447497,4.324514,-3.754064,-2.713567,-1.115429,-2.905486,16.718457,9.635833,-12.775944,-13.233410,-3.275423,-2.742340,1.176753,9.979324,4.363920,13.449135,-4.677430,9.809426,3.883498,11.865272,-23.622502,...,6.133633,8.453127,2.723381,6.025675,-28.799432,5.911213,0.715092,26.158372,3.577201,-2.053691,2.117856,-4.933909,-16.165107,-7.292672,-4.659052,-2.085703,3.761888,13.230259,1.065335,0.559134,9.310662,-3.011264,3.997825,1.174759,-13.503914,2.716392,-4.664858,3.238620,3.586218,-17.810438,7.231024,1.531186,-7.528823,0.473802,-11.864658,-11.293788,1.866265,3.616046,11.971096,0
1,4.649688,9.974928,20.330103,-0.440372,-18.128566,-1.984671,4.906458,-27.797837,-21.262646,96.965297,-16.834383,-8.287409,0.276578,-5.521477,10.737403,11.559558,12.361678,21.471950,1.469455,-3.359539,24.220936,2.367283,-7.231356,0.065299,27.347943,-2.464829,-21.663887,-5.925950,-22.747166,24.290036,-0.988310,-7.474970,-12.236707,16.085088,-14.693179,10.205569,-8.787829,-9.007780,-11.395117,-7.176609,...,-3.910526,7.288979,2.170080,-7.279955,-18.213181,6.770618,9.148779,-26.160896,8.483096,-31.722966,6.469263,2.626295,23.202933,3.972109,15.887269,-2.181704,4.839923,-16.290227,6.290554,-7.516897,-10.104270,0.889068,4.639309,-13.386870,-4.264800,-0.211778,-1.539722,-3.005059,1.201008,23.015827,3.435464,-5.169600,7.102491,34.516881,6.177686,-27.770856,12.926435,-4.564559,33.919834,1
2,-17.305105,17.355614,-9.135664,-6.038550,-1.831651,4.547895,17.935764,-4.799029,3.100311,99.380095,-11.162563,-11.754073,-1.744995,15.092583,-7.351360,-5.347104,-11.513937,88.115309,-7.072646,-10.919763,1.670091,-4.712406,-9.395305,-8.954009,-1.067681,-7.232015,-0.895293,-0.489190,2.165871,3.981562,6.955223,4.807198,-4.357221,3.463160,-1.398599,-9.892089,-12.493803,17.392627,12.400411,-12.010533,...,7.617686,1.602701,-7.681268,-1.707590,-11.960032,-11.658661,10.529848,40.387187,-7.645222,2.593719,2.098322,-3.837943,-2.898191,2.481844,5.720931,-14.590002,-4.597881,9.480376,9.611217,-0.373726,2.544593,-0.096869,18.105063,7.752510,-13.112805,3.139476,-2.465182,15.322524,-5.929012,-24.310109,-1.216773,11.909693,9.591573,11.846737,1.397859,6.454157,-0.271460,-12.500337,27.634567,2
3,3.897911,2.545857,-2.053792,3.385450,3.424216,-2.282545,-11.763825,6.692485,5.797674,94.978085,-9.535053,7.275684,-7.260567,-3.684431,0.619918,-11.668374,-2.629462,47.033020,-3.705021,0.252289,-2.764885,-7.615028,-2.610788,8.418192,-1.816173,-4.397300,-7.041045,1.485299,-0.500037,-2.161665,-2.521797,4.674520,-1.450020,-0.026549,-2.202821,-1.435662,0.090911,-2.340973,-8.300779,-6.890711,...,-2.087713,-9.613727,3.222594,8.271757,-4.114560,3.857648,-0.776361,7.146503,-7.063720,5.565487,-5.565373,-4.886646,-17.171175,-5.723995,-3.184984,-5.072432,3.335052,-0.249573,0.504552,11.300645,6.992478,-5.459367,-3.192625,0.492273,-0.089809,1.211323,-3.443094,5.896227,0.834543,-5.435584,1.672591,-0.863278,-2.906553,-3.466688,-3.867892,-4.249463,-12.551012,4.494087,-6.223341,3
4,-5.391206,1.767221,1.810128,-4.097073,-3.623262,8.417368,-25.246265,7.473430,-2.789541,89.594627,-19.181812,-2.545340,1.786155,7.171871,-6.570599,4.070353,-3.981817,64.771924,-7.667751,-4.576751,2.692816,3.660733,-6.979315,-2.746959,-3.971580,-5.004535,-1.527362,0.767176,2.057152,-8.098983,-12.731448,-2.521009,-3.315749,-27.243848,3.714960,-2.194191,9.102283,-2.869839,5.023587,8.512611,...,-0.929634,-1.385753,-4.337329,11.370485,-0.861715,-2.491243,-3.582624,-1.165465,-0.898303,19.585878,-6.021031,0.980944,4.945259,-5.333400,-4.065658,-10.818530,7.630075,-5.085632,12.288123,7.125362,-2.053

In [0]:
if not os.path.isfile('/content/drive/My Drive/Colab Notebooks/Models/Quora/final_features.parquet.gzip'):
  txt_ft_q1['id'] = nlp_feat['id']
  txt_ft_q2['id'] = nlp_feat['id']
  #Merging nlp features & preprocessing features
  nlp_feat = nlp_feat.merge(ppro_feat, on='id', how='left')
  #Merging txt_feat of q1&q2
  txt_feat = txt_ft_q1.merge(txt_ft_q2, on='id', how='left')
  #Merging nlp_feat & txt_feat
  final_features = nlp_feat.merge(txt_feat, on='id', how ='left')
  #Saving only the features into a feather file
  final_features.to_parquet('/content/drive/My Drive/Colab Notebooks/Models/Quora/final_features.parquet.gzip',compression='gzip')
else:
  final_features = pd.read_parquet('/content/drive/My Drive/Colab Notebooks/Models/Quora/final_features.parquet.gzip', engine = 'fastparquet')

In [0]:
final_features.head()

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio,freq_qid1,freq_qid2,len_q1,len_q2,n_words_q1,n_words_q2,words_Common,words_Total,word_Share,freq_q1+q2,freq_q1-q2,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,...,260_y,261_y,262_y,263_y,264_y,265_y,266_y,267_y,268_y,269_y,270_y,271_y,272_y,273_y,274_y,275_y,276_y,277_y,278_y,279_y,280_y,281_y,282_y,283_y,284_y,285_y,286_y,287_y,288_y,289_y,290_y,291_y,292_y,293_y,294_y,295_y,296_y,297_y,298_y,299_y
0,0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.916659,0.0,1.0,2.0,13.0,100,93,93,100,0.982456,1,1,66,57,14,12,10.0,10.0,0.434783,2,2,-5.856872,17.449559,4.862720,7.971019,20.345586,-5.514759,-4.077800,-2.820742,8.029026,146.599092,-45.501892,-3.234358,...,-1.490777,6.133633,8.453127,2.723381,6.025675,-28.799432,5.911213,0.715092,26.158372,3.577201,-2.053691,2.117856,-4.933909,-16.165107,-7.292672,-4.659052,-2.085703,3.761888,13.230259,1.065335,0.559134,9.310662,-3.011264,3.997825,1.174759,-13.503914,2.716392,-4.664858,3.238620,3.586218,-17.810438,7.231024,1.531186,-7.528823,0.473802,-11.864658,-11.293788,1.866265,3.616046,11.971096
1,1,0,0.799984,0.399996,0.749981,0.749981,0.699993,0.699993,0.0,1.0,5.0,12.5,86,63,65,73,0.571429,4,1,51,88,8,13,4.0,4.0,0.200000,5,5,9.356103,13.098566,18.945098,-2.079594,-15.703841,-2.173409,8.969065,-20.458267,-20.674299,13.760798,-4.915949,-12.053218,...,3.382111,-3.910526,7.288979,2.170080,-7.279955,-18.213181,6.770618,9.148779,-26.160896,8.483096,-31.722966,6.469263,2.626295,23.202933,3.972109,15.887269,-2.181704,4.839923,-16.290227,6.290554,-7.516897,-10.104270,0.889068,4.639309,-13.386870,-4.264800,-0.211778,-1.539722,-3.005059,1.201008,23.015827,3.435464,-5.169600,7.102491,34.516881,6.177686,-27.770856,12.926435,-4.564559,33.919834
2,2,0,0.399992,0.333328,0.399992,0.399992,0.399996,0.399996,0.0,1.0,4.0,12.0,65,64,43,46,0.172414,1,1,73,59,14,10,4.0,4.0,0.166667,2,2,0.909520,16.050299,-8.126856,-4.848289,-2.806190,9.752280,4.349992,-5.120332,6.785252,106.342974,-12.946087,-4.638530,...,21.557987,7.617686,1.602701,-7.681268,-1.707590,-11.960032,-11.658661,10.529848,40.387187,-7.645222,2.593719,2.098322,-3.837943,-2.898191,2.481844,5.720931,-14.590002,-4.597881,9.480376,9.611217,-0.373726,2.544593,-0.096869,18.105063,7.752510,-13.112805,3.139476,-2.465182,15.322524,-5.929012,-24.310109,-1.216773,11.909693,9.591573,11.846737,1.397859,6.454157,-0.271460,-12.500337,27.634567
3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,4.0,9.0,29,28,26,32,0.078947,1,1,50,65,11,9,0.0,0.0,0.000000,2,2,-4.950745,17.098874,-15.474965,1.044680,-2.392017,-0.051889,2.650595,-8.451192,2.584123,116.184408,-9.716004,-5.455535,...,5.902119,-2.087713,-9.613727,3.222594,8.271757,-4.114560,3.857648,-0.776361,7.146503,-7.063720,5.565487,-5.565373,-4.886646,-17.171175,-5.723995,-3.184984,-5.072432,3.335052,-0.249573,0.504552,11.300645,6.992478,-5.459367,-3.192625,0.492273,-0.089809,1.211323,-3.443094,5.896227,0.834543,-5.435584,1.672591,-0.863278,-2.906553,-3.466688,-3.867892,-4.249463,-12.551012,4.494087,-6.223341
4,4,0,0.199996,0.099999,0.999950,0.999950,0.428565,0.428565,0.0,1.0,6.0,10.0,53,44,39,53,0.162162,3,1,76,39,13,7,2.0,2.0,0.100000,4,4,-11.520302,19.769948,-4.510997,-6.548994,-20.835286,33.663909,-30.390504,0.826553,-19.571472,84.458577,-44.555112,-1.895740,...,8.020295,-0.929634,-1.385753,-4.337329,11.370485,-0.861715,-2.491243,-3.582624,-1.165465,-0.898303,19.585878,-6.021031,0.980944,4.945259,-5.333400,-4.065658,-10.818530,7.630075,-5.085632,12.288123,7.125362,-2.053170,-5.169374,-7.881771,-0.176188,6.449292,22.000222,-3.097348,6.809265,1.921036,-10.407441,-8.444207,-14.450059,-12.709382,-4.449050,12.563987,-11.721362,-16.459300,3.626297,-9.790615
